In [118]:
# Static network
import dynet as dy

# multilayer perceptron with a single hidden layer
# input 2 nodes
# hidden layer: 8 nodes, activation: tanh
# output layer: 1 node
# σ(V(tanh(Wx+b)))
# x: 2x1
# W: 8x2
# b: 8 vector
# V: 1x8

# define the parameters
m = dy.ParameterCollection()
pW = m.add_parameters((8,2)) # _dynet.Parameters
pV = m.add_parameters((1,8))
pb = m.add_parameters((8))

In [119]:
dy.renew_cg() # new computation graph

In [120]:
# add the parameters to the graph
# create Expression representing the network 
#(the network will include the Expressions 
# for the Parameters defined in the parameter collection)
W = dy.parameter(pW) #W is of type _dynet.Expression
V = dy.parameter(pV)
b = dy.parameter(pb)

In [121]:
print('W.dimension: {}'.format(W.dim()))
print('b.dimension: {}'.format(b.dim()))
print('V.dimension: {}'.format(V.dim()))

W.dimension: ((8, 2), 1)
b.dimension: ((8,), 1)
V.dimension: ((1, 8), 1)


In [122]:
# create the network
x = dy.vecInput(2) # an input vector of size 2. _dynet._vecInputExpression
output = dy.logistic(V*(dy.tanh((W*x)+b))) # output is _dynet.Expression

In [123]:
print(type(x))
print(type(output))

<class '_dynet._vecInputExpression'>
<class '_dynet.Expression'>


In [124]:
print((W*x+b).dim())
print(dy.tanh((W*x+b)).dim())
print((V*dy.tanh((W*x+b))).dim())
print(dy.logistic(V*dy.tanh((W*x+b))).dim())
print(output.dim())

((8,), 1)
((8,), 1)
((1,), 1)
((1,), 1)
((1,), 1)


In [125]:
# we can now query our network
x.set([0,0])
output.value()

0.6098086833953857

In [126]:
# we want to be able to define a loss, so we need an input expression to work against.
y = dy.scalarInput(0) # this will hold the correct answer
loss = dy.binary_log_loss(output, y)

In [127]:
print(type(y))
print(type(loss))

<class '_dynet._inputExpression'>
<class '_dynet.Expression'>


In [128]:
# trainer = dy.SimpleSGDTrainer(m)
# trainer = dy.CyclicalSGDTrainer(m)
# trainer = dy.AdamTrainer(m)
# trainer = dy.AdagradTrainer(m)
trainer = dy.RMSPropTrainer(m)

In [129]:
type(trainer)

_dynet.RMSPropTrainer

In [130]:
trainer.learning_rate

0.0010000000474974513

In [131]:
# single step optimization/training

x.set([1,0])
y.set(1)
loss_value = loss.value() # this performs a forward through the network.
print("the loss before step is:",loss_value)

# now do an optimization step
loss.backward()  # compute the gradients
trainer.update()

# see how it affected the loss:
loss_value = loss.value(recalculate=True) # recalculate=True means "don't use precomputed value"
print("the loss after step is:",loss_value)

the loss before step is: 0.24690616130828857
the loss after step is: 0.24105413258075714


In [132]:
# create training data set
def create_xor_instances(num_rounds=2000):
    questions = []
    answers = []
    for round in range(num_rounds):
        for x1 in 0,1:
            for x2 in 0,1:
                answer = 0 if x1==x2 else 1
                questions.append((x1,x2))
                answers.append(answer)
    return questions, answers

questions, answers = create_xor_instances()

In [133]:
print(len(questions))
print(len(answers))
print(questions[:4])
print(answers[:4])

8000
8000
[(0, 0), (0, 1), (1, 0), (1, 1)]
[0, 1, 1, 0]


In [134]:
# training
total_loss = 0
seen_instances = 0
for question, answer in zip(questions, answers):
    x.set(question)
    y.set(answer)
    seen_instances += 1
    total_loss += loss.value() # forward
    loss.backward()
    trainer.update()
    if (seen_instances > 1 and seen_instances % 100 == 0):
        print("average loss is:",total_loss / seen_instances)

average loss is: 0.7972901383042336
average loss is: 0.7745968557894229
average loss is: 0.7570578746000926
average loss is: 0.7432997459173203
average loss is: 0.7323096290230751
average loss is: 0.7233400035897891
average loss is: 0.715852968096733
average loss is: 0.7094636897742749
average loss is: 0.70389404296875
average loss is: 0.6989397900104523
average loss is: 0.6944485448707234
average loss is: 0.6903049652278423
average loss is: 0.6864206374608554
average loss is: 0.6827270120382309
average loss is: 0.6791703767776489
average loss is: 0.6757082035019993
average loss is: 0.6723064525337781
average loss is: 0.668937573366695
average loss is: 0.6655790155498605
average loss is: 0.6622121162116528
average loss is: 0.6588212782854126
average loss is: 0.6553933482820338
average loss is: 0.6519171639629032
average loss is: 0.6483832151691119
average loss is: 0.6447833909749985
average loss is: 0.6411107977766257
average loss is: 0.6373596195269514
average loss is: 0.6335250169783

In [135]:
x.set([0,1])
print("0,1",output.value())

x.set([1,0])
print("1,0",output.value())

x.set([0,0])
print("0,0",output.value())

x.set([1,1])
print("1,1",output.value())

0,1 0.9514448046684265
1,0 0.9690647125244141
0,0 0.012001067399978638
1,1 0.05876074358820915


In [115]:
W.value()

array([[ 4.07663822, -2.77607584],
       [ 2.17588735, -3.42900205],
       [ 3.63992882,  3.72946382],
       [ 0.06008036, -1.52189255],
       [-3.54680562,  2.17219925],
       [ 1.74713027, -2.989784  ],
       [-0.82024181, -0.7913931 ],
       [ 4.60517788,  4.43824816]])

In [96]:
V.value()

array([[-1.26245904,  0.91464269,  1.09329867,  0.70665795,  1.29633462,
         1.35833979,  0.8188495 ,  1.72384453]])

In [97]:
b.value()

[0.6605279445648193,
 -0.45908284187316895,
 -0.1739916205406189,
 0.2111283540725708,
 -0.35437655448913574,
 -0.33467215299606323,
 0.3537399470806122,
 -0.7486727237701416]

In [31]:
m.save('models/xor_static_model')